pagina de referencia: https://datascienceparichay.com/article/jaccard-similarity-python/

In [1]:
import pandas as pd
import json
from collections import defaultdict
import numpy as np

In [2]:
def jaccard_similarity(A, B):
    # Convert the sets to frozensets for better performance
    set_A = frozenset(A)
    set_B = frozenset(B)

    # Calculate the intersection size
    intersection = len(set_A & set_B)

    # Calculate the union size
    union = len(set_A | set_B)

    # Take the ratio of sizes
    if union == 0:
        return 0
    similarity = intersection / union

    return similarity

In [3]:
df = pd.read_csv('tweets_2022_abril_junio.csv') # cuidado al ejecutar

In [4]:
df.drop(['id', 'created_at', 'retweet_count', 'favorite_count'], axis=1, inplace=True)

In [5]:
df.head()

,screen_name,text
0,h0l4d4ni3l4,RT @ValeMirandaCC: Tras casi 50 años del golpe...
1,Claudio70932894,RT @UTDTrabajoDigno: Mañana jueves a las 18hrs...
2,Cesar_A_RR,RT @JaimeGuajardoR: Aquí está el aporte de @te...
3,rosmarieher,RT @melnicksergio: la pelotudez no tiene limit...
4,GQuelluen,RT @BSepulvedaHales: Ante la circulación de no...


In [6]:
df.shape

(4594980, 2)

In [7]:
df['text'][0][:4]

'RT @'

In [8]:
tweets_originales = df.loc[df['text'].str[:4].str.contains('RT @',case=False) == False]

rt = df.loc[df['text'].str[:4].str.contains('RT @',case=False) == True]

In [9]:
tweets_originales.head()

,screen_name,text
6,simonlodijo,@unveranonaranja @ruidosafest @franciscamusic ...
9,MacaSimplemente,@LaSuRivas @ElisaLoncon @siliconvalle @Valdebe...
12,LuisVer75699645,@teanval0207 @izkia @arturozunigaj Excelente...
13,MITERRED,@mcubillossigall https://t.co/gkg5PwbZhZ
14,piaelizabethvm,@simonlodijo @ruidosafest @franciscamusic @gio...


In [10]:
rt.head()

,screen_name,text
0,h0l4d4ni3l4,RT @ValeMirandaCC: Tras casi 50 años del golpe...
1,Claudio70932894,RT @UTDTrabajoDigno: Mañana jueves a las 18hrs...
2,Cesar_A_RR,RT @JaimeGuajardoR: Aquí está el aporte de @te...
3,rosmarieher,RT @melnicksergio: la pelotudez no tiene limit...
4,GQuelluen,RT @BSepulvedaHales: Ante la circulación de no...


In [11]:
retweets = pd.DataFrame()

split_text = rt['text'].str.split(' ', expand=True, n=2)
retweets['user'] = split_text[1].str.strip("@").str.strip(':')
retweets['tweet'] = split_text[2]

retweets.reset_index(drop=True, inplace=True)

retweets

,user,tweet
0,ValeMirandaCC,"Tras casi 50 años del golpe, la Constitución s..."
1,UTDTrabajoDigno,Mañana jueves a las 18hrs. comienza nuestro pr...
2,JaimeGuajardoR,Aquí está el aporte de @tere_marinovic con res...
3,melnicksergio,la pelotudez no tiene limites...no tiene
4,BSepulvedaHales,"Ante la circulación de noticias falsas, les qu..."
...,...,...
3327415,DanielAbelLope1,@tere_marinovic 😡🤮😡🤮 VIEJO C.S.M. 🤮😡🤮 https://...
3327416,DanielAbelLope1,@tere_marinovic 😡🤮😡🤮 VIEJO C.S.M. 🤮😡🤮 https://...
3327417,Gonz1Gorjeperez,@tere_marinovic https://t.co/OWoyaR8Dch
3327418,Gonz1Gorjeperez,@tere_marinovic https://t.co/XWq7hJ3kyv


In [12]:
retweets.head()

,user,tweet
0,ValeMirandaCC,"Tras casi 50 años del golpe, la Constitución s..."
1,UTDTrabajoDigno,Mañana jueves a las 18hrs. comienza nuestro pr...
2,JaimeGuajardoR,Aquí está el aporte de @tere_marinovic con res...
3,melnicksergio,la pelotudez no tiene limites...no tiene
4,BSepulvedaHales,"Ante la circulación de noticias falsas, les qu..."


In [13]:
tweets_originales.rename(columns={'screen_name': 'user', 'text': 'tweet'}, inplace=True)

C:\Users\jor_x\AppData\Local\Temp\ipykernel_14592\2716881129.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_originales.rename(columns={'screen_name': 'user', 'text': 'tweet'}, inplace=True)


In [14]:
tweets = pd.concat([tweets_originales, retweets]).reset_index(drop=True)

tweets

,user,tweet
0,simonlodijo,@unveranonaranja @ruidosafest @franciscamusic ...
1,MacaSimplemente,@LaSuRivas @ElisaLoncon @siliconvalle @Valdebe...
2,LuisVer75699645,@teanval0207 @izkia @arturozunigaj Excelente...
3,MITERRED,@mcubillossigall https://t.co/gkg5PwbZhZ
4,piaelizabethvm,@simonlodijo @ruidosafest @franciscamusic @gio...
...,...,...
4594975,DanielAbelLope1,@tere_marinovic 😡🤮😡🤮 VIEJO C.S.M. 🤮😡🤮 https://...
4594976,DanielAbelLope1,@tere_marinovic 😡🤮😡🤮 VIEJO C.S.M. 🤮😡🤮 https://...
4594977,Gonz1Gorjeperez,@tere_marinovic https://t.co/OWoyaR8Dch
4594978,Gonz1Gorjeperez,@tere_marinovic https://t.co/XWq7hJ3kyv


In [15]:
final_tweets = tweets.drop_duplicates(['user', 'tweet'], ignore_index=True)

final_tweets

,user,tweet
0,simonlodijo,@unveranonaranja @ruidosafest @franciscamusic ...
1,MacaSimplemente,@LaSuRivas @ElisaLoncon @siliconvalle @Valdebe...
2,LuisVer75699645,@teanval0207 @izkia @arturozunigaj Excelente...
3,MITERRED,@mcubillossigall https://t.co/gkg5PwbZhZ
4,piaelizabethvm,@simonlodijo @ruidosafest @franciscamusic @gio...
...,...,...
1442712,hocikonapatriot,@tere_marinovic fuiste la voz de muchos chilen...
1442713,Caroline_walls6,@tere_marinovic Si la Tere pudiera decir lo qu...
1442714,EduardoAlegri16,@tere_marinovic Justamente la mayor ausente e...
1442715,AlonsoRPatriota,"@maluchayallego Historica debería ser, el dar ..."


In [16]:
tweet_shingles = defaultdict(set)
inverted_index = defaultdict(set)
k = 5
batch_size = 100000
cantidad = final_tweets.shape[0]

for carga in range(0, cantidad, batch_size):
    batch_end = min(carga + batch_size, cantidad)
    batch_tweets = final_tweets.iloc[carga:batch_end]

    for _, tweet in batch_tweets.iterrows():
        user = tweet['user']
        tweet_text = tweet['tweet']

        shingles = set(tweet_text[i:i+k] for i in range(len(tweet_text) - k + 1))
        tweet_shingles[user].update(shingles)

In [17]:
len(tweet_shingles.keys())

131799

In [39]:
# shingle_dict = {}  # Diccionario para almacenar los mapeos de shingles a identificadores
# next_id = 0  # Siguiente identificador disponible

# # Recorrer los usuarios y sus shingles
# for shingles in tweet_shingles.values():
#     for shingle in shingles:
#         if shingle not in shingle_dict:
#             # Si el shingle no está en el diccionario, asignarle un nuevo identificador
#             shingle_dict[shingle] = next_id
#             next_id += 1

# # Ahora, shingle_dict contiene los mapeos de shingles a identificadores únicos


In [40]:
# usuarios = list(tweet_shingles.keys())

# # Parámetros
# num_permutations = 1000  # Número de permutaciones para MinHash

# # Crear matriz de firma MinHash
# signature_matrix = np.zeros((len(usuarios), num_permutations), dtype=np.uint32)

# # Generar permutaciones aleatorias
# permutations = np.random.permutation(len(tweet_shingles)).tolist()

# # Calcular firma MinHash para cada usuario
# for i, user in enumerate(usuarios):
#     for j, permutation in enumerate(permutations[:num_permutations]):
#         shingles = [shingle_dict[shingle] for shingle in tweet_shingles[user] if shingle in shingle_dict]
#         if shingles:
#             signature_matrix[i, j] = min(shingles)
#         else:
#             signature_matrix[i, j] = 0  # O cualquier otro valor por defecto que desees asignar cuando no haya shingles presentes




In [18]:
from datasketch import MinHashLSHForest, MinHash

# Crear el índice LSH
forest = MinHashLSHForest(num_perm=128)  # Ajusta el número de permutaciones según tus necesidades

# Agregar los MinHash de los usuarios al índice LSH
for user, shingles in tweet_shingles.items():
    minhash = MinHash(num_perm=128)  # Ajusta el número de permutaciones según tus necesidades
    
    for shingle in shingles:
        minhash.update(shingle.encode('utf-8'))
    
    forest.add(user, minhash)

# Construir el índice LSH
forest.index()

# Definir el umbral de similaridad de Jaccard
threshold = 0.8

# Encontrar usuarios similares utilizando LSH
user_similares = defaultdict(set)

for user, shingles in tweet_shingles.items():
    minhash_query = MinHash(num_perm=128)  # Ajusta el número de permutaciones según tus necesidades
    
    for shingle in shingles:
        minhash_query.update(shingle.encode('utf-8'))
    
    # Consultar el índice LSH para encontrar los usuarios similares
    result = forest.query(minhash_query, k=4)  # Ajusta el valor de 'k' según tus necesidades
    
    for similar_user in result:
        if similar_user != user:  # Excluir el mismo usuario de la lista de usuarios similares
            similarity = jaccard_similarity(tweet_shingles[user], tweet_shingles[similar_user])
            if threshold <= similarity < 1:
                user_similares[user].add((similar_user, similarity))


In [21]:
jaccard_similarity(tweet_shingles['moyarcec'], tweet_shingles['Cathy_montecino'])

0.9047619047619048

In [26]:
tweet_shingles['Cathy_montecino']

{' Esta',
 ' llen',
 '@Jaim',
 'Bassa',
 'Esta ',
 'Jaime',
 '_Bass',
 'a Est',
 'a lle',
 'aime_',
 'assa ',
 'e_Bas',
 'eno 😩',
 'ime_B',
 'leno ',
 'lleno',
 'me_Ba',
 'sa Es',
 'ssa E',
 'sta l',
 'ta ll'}

In [20]:
user_similares

defaultdict(set,
            {'moyarcec': {('Cathy_montecino', 0.9047619047619048),
              ('carmencamalvare', 0.95),
              ('doblon_tuyo', 0.9047619047619048)},
             'ojosdebestia': {('AnnRossovich', 0.8571428571428571)},
             'PepVenturi': {('lslvmy', 0.8076923076923077)},
             'AndresF69637799': {('antoreynaldos', 0.9047619047619048)},
             'FefyMon': {('juanalexsegovia', 0.8)},
             'solange_azocar': {('f044a1d2f17b49b', 0.875)},
             'carmencamalvare': {('Cathy_montecino', 0.8636363636363636),
              ('doblon_tuyo', 0.8636363636363636),
              ('moyarcec', 0.95)},
             'Claudioxirola': {('RobHellraiser', 0.9411764705882353),
              ('WSteinbrecherAr', 0.8823529411764706)},
             'Jorge60119837': {('DnieldelValle', 0.8666666666666667),
              ('Elojesa', 0.8666666666666667)},
             'juankalva': {('Rubendariosalas', 0.8)},
             'Coralinetapia': {('AleSmith03', 0.9

In [30]:
final_tweets.loc[final_tweets['user'] == 'doblon_tuyo']

,user,tweet
234792,doblon_tuyo,@Jaime_Bassa Esta lleno 😭


In [48]:
final_tweets.loc[final_tweets['user'] == 'Gonzalo_Celedon']

,user,tweet
31020,Gonzalo_Celedon,#ConMiPlataNo
103799,Gonzalo_Celedon,Ojalá así sea\n#RechazoDeSalida\n#ConMiPlataNo
199855,Gonzalo_Celedon,@YarelaAysen @danielstingo Entonces es mentira...
206701,Gonzalo_Celedon,¿Plurinacional?\n\n#ConMiPlataNo
426119,Gonzalo_Celedon,@ConstituyenteFA @danielstingo Entonces es men...


In [42]:
user_similares

defaultdict(set,
            {'KENALATEJEDORA': {('jannyoy', 1.0), ('primor2', 1.0)},
             'paitogutierrez': {('Gonzalo_Celedon', 1.0), ('loreym86', 1.0)},
             'NahirNavarro1': {('Ingrid_A6', 1.0),
              ('LavadoPame', 1.0),
              ('cpmare', 1.0)},
             'drummondvictor': {('CLisboa5', 1.0),
              ('Sr_Rinco', 1.0),
              ('bustos396', 1.0)},
             'moyarcec': {('carmencamalvare', 0.95), ('hfuentesmtz', 1.0)},
             'jorgebriones58': {('fmdiazlemus', 0.8571428571428571)},
             'vilolo2010': {('IIWilfredII', 0.9411764705882353),
              ('lapolillaloca19', 0.9411764705882353),
              ('manguakin1979', 0.9411764705882353)},
             'Ogu19592010': {('MiyakiGeork', 0.8636363636363636),
              ('edyrivera74', 1.0),
              ('javier83404094', 0.8636363636363636)},
             'AdriSanMed': {('nanu_sc22', 0.8666666666666667)},
             'ThomasJ77570396': {('Carolagiaconi', 0.93333

In [17]:
user_similares = defaultdict(set)
usuarios = list(tweet_shingles.keys())

largo = len(tweet_shingles.keys())

for i in range(largo-1):
    user = usuarios[i]
    for j in range(i+1, largo):
        user_2 = usuarios[j]
        similarity = jaccard_similarity(tweet_shingles[user], tweet_shingles[user_2])
        if 0.88 <= similarity < 1:
            user_similares[user].add(user_2)
        

In [18]:
user_similares

defaultdict(set,
            {'moyarcec': {'carmencamalvare'},
             'CleyRaw': {'antmonda'},
             'vilolo2010': {'lapolillaloca19'},
             'pablo_pitt27': {'vhkiosko'},
             'Veronic62782080': {'consignatario'},
             'rodrigopachecor': {'MaraBarrientos4'},
             'stay_naive': {'wuanglen8'},
             'cesarciffo': {'Isacar76265142'},
             'CharlyWolf4': {'pazuag'},
             'MairaSZV': {'hernyp45'},
             'VStuckrath': {'Lore1415A'}})

In [25]:
filtered_user = {}

for k, v in user_similares.items():
    if len(v) > 0:
        filtered_user[k] = v

In [26]:
filtered_user

{'moyarcec': {'carmencamalvare'},
 'CleyRaw': {'antmonda'},
 'vilolo2010': {'lapolillaloca19'},
 'pablo_pitt27': {'vhkiosko'},
 'Veronic62782080': {'consignatario'},
 'rodrigopachecor': {'MaraBarrientos4'},
 'stay_naive': {'wuanglen8'},
 'cesarciffo': {'Isacar76265142'},
 'CharlyWolf4': {'pazuag'},
 'MairaSZV': {'hernyp45'},
 'VStuckrath': {'Lore1415A'}}

In [ ]:
# user_similares = defaultdict(set)

# users = list(tweet_shingles.keys())
# n_users = len(users)

# for i in range(n_users):
#     user = users[i]
#     shingles = tweet_shingles[user]

#     for j in range(i + 1, n_users):
#         user_2 = users[j]
#         shingles_2 = tweet_shingles[user_2]

#         similarity = jaccard_similarity(shingles, shingles_2)
#         if 0.75 <= similarity < 1:
#             user_similares[user].add(user_2)
#             user_similares[user_2].add(user)

In [22]:
user_similares['RuthSalinasGal']

set()

In [23]:
final_tweets.loc[final_tweets['user'] == 'VStuckrath']

,user,tweet
7919,VStuckrath,@Jaime_Bassa Gracias por avisar para no verlo
253031,VStuckrath,@tere_marinovic Calladita no más por que si h...
634781,VStuckrath,@ChunchoXelMundo @ElisaLoncon Yo ya no se que ...
793219,VStuckrath,@Sebasti45781359 @cata_1933 @cmonckeberg Llama...
953858,VStuckrath,@RenSHevia Bienvenido al #RechazoCrece #Rechaz...
994923,VStuckrath,@Alejand64086478 @paulylm @labeasanchez Todo e...
1022602,VStuckrath,@RafaelTorrebl18 @Edoandrew4 @convencioncl @fe...
1064632,VStuckrath,@panchozen @tere_marinovic Subirán aún más lo ...
1286187,VStuckrath,@BRebolledoD17 @bobbybo73 @patricionavia @tere...
1415546,VStuckrath,@tere_marinovic @cretton15 Bien ahí... sigue l...


In [24]:
final_tweets.loc[final_tweets['user'] == 'Lore1415A']

,user,tweet
7931,Lore1415A,@Jaime_Bassa Gracias por avisar para no verlo.
41439,Lore1415A,@majive3105 @yasnaguzman @Alexisgomeza21 @Ampu...
42325,Lore1415A,@majive3105 @yasnaguzman @Alexisgomeza21 @Ampu...
98558,Lore1415A,@majive3105 @AmpueroAdriana Hablas de mi gener...
156524,Lore1415A,"@majive3105 @Skobar1961 @AmpueroAdriana Y si, ..."
176871,Lore1415A,@IgnacioAchurra No ha pedido disculpas pública...
216360,Lore1415A,@yasnaguzman @Alexisgomeza21 @majive3105 @Ampu...
217594,Lore1415A,@majive3105 @yasnaguzman @Alexisgomeza21 @Ampu...
260870,Lore1415A,@tere_marinovic Vamos por el rechazo de salida...
274796,Lore1415A,@majive3105 @Skobar1961 @AmpueroAdriana La mis...


In [25]:
# tweet_shingles = dict()
# inverted_index = dict()
# k=5

# for id in range(10000): 
  
#     tweet_shingles[id] = set()
    
#     for i in range(len(final_tweets['tweet'][id]) - k-1):        
#         shingle = final_tweets['tweet'][id][i:i+k]
#         tweet_shingles[id].add(shingle)

#         if shingle not in inverted_index:
#             inverted_index[shingle] = set()
#         inverted_index[shingle].add((id, final_tweets.iloc[id]['user']))

#     tweet_shingles[id] = list(tweet_shingles[id])

In [26]:
# jaccard_user = dict()

# for id in tweet_shingles.keys():
#     user_1 = final_tweets.iloc[id]['user']
    
#     for id_2 in tweet_shingles.keys():
#         user_2 = final_tweets.iloc[id_2]['user']
        
#         if user_1 != user_2:
#             shingle_similarity = jaccard_similarity(tweet_shingles[id], tweet_shingles[id_2])
            
#             if 0.8 <= shingle_similarity < 1:
                
#                 if user_1 not in jaccard_user:
#                     jaccard_user[user_1] = dict()
                    
#                 if user_2 not in jaccard_user[user_1]:
#                     jaccard_user[user_1][user_2] = [0, 0]
                    
#                 jaccard_user[user_1][user_2][1] += 1
#                 jaccard_user[user_1][user_2][0] += shingle_similarity

In [27]:
filtered_index = dict()

for k, v in inverted_index.items():
    if len(v) > 1:
        filtered_index[k] = v
        
filtered_index

{'unver': {(0, 'simonlodijo'), (5789, 'PorricoElza')},
 'nvera': {(0, 'simonlodijo'), (5789, 'PorricoElza')},
 'veran': {(0, 'simonlodijo'),
  (1906, '__JVP__'),
  (4368, 'jarcecontre'),
  (5446, 'Alejand88202848'),
  (5789, 'PorricoElza'),
  (7168, 'CamiOlZu97')},
 'erano': {(0, 'simonlodijo'),
  (580, 'don_pernil2'),
  (5148, 'Manquehuito2'),
  (5789, 'PorricoElza')},
 'ranon': {(0, 'simonlodijo'), (5789, 'PorricoElza')},
 'anona': {(0, 'simonlodijo'), (5789, 'PorricoElza')},
 'nonar': {(0, 'simonlodijo'), (5789, 'PorricoElza')},
 'onara': {(0, 'simonlodijo'), (5789, 'PorricoElza')},
 'naran': {(0, 'simonlodijo'),
  (4309, 'paulabv11'),
  (4554, 'Caroexpress'),
  (5789, 'PorricoElza'),
  (7005, 'sifuenteschile')},
 'aranj': {(0, 'simonlodijo'),
  (4309, 'paulabv11'),
  (4554, 'Caroexpress'),
  (5789, 'PorricoElza'),
  (7005, 'sifuenteschile'),
  (8344, 'Bnjmin__'),
  (8741, 'TOiuho_')},
 'ranja': {(0, 'simonlodijo'),
  (2138, 'nubegrumosa'),
  (3454, 'Fevecris'),
  (5789, 'PorricoElz

In [28]:
# file = "shingles.json"

# with open(file, "w") as file:
#     json.dump(tweet_shingles, file)

In [29]:
# filtered_inverted_index = dict()

# for k, v in inverted_index.items():
#     if len(v) > 1 and not list(v) in filtered_inverted_index.values():
#         filtered_inverted_index[k] = list(v)

# file = "filtered_inverted_index.json"

# with open(file, "w") as file:
#     json.dump(filtered_inverted_index, file)

In [30]:
# file = "shingles.json"

# with open(file, "r") as file:
#     tweet_shingles = json.load(file)

In [31]:
# file = "filtered_inverted_index.json"

# with open(file, "r") as file:
#     filtered_inverted_index = json.load(file)

In [32]:
# similar = []
# s = 0.4

# for pair in filtered_inverted_index.values():
#     if len(pair) == 2:
#         tweet1 = tweet_shingles[str(pair[0])]
#         tweet2 = tweet_shingles[str(pair[1])]

#         sim = jaccard_similarity(tweet1, tweet2)
#         if sim > s:
#             similar.append((pair[0], pair[1]))